# CS 375 Homework 2: Naive Bayes for Text Classification
The goal of this assignment is to give you practice implementing a Naive Bayes classifier and evaluate the classifier's performance on real-world datasets. 

You'll apply your Naive Bayes classifier to two datasets: messages for disaster relief (`triage`) and sentiment about COVID-19 (`covid`).

## Organization and Instructions 

Execute the code cells in Part 1 to understand the background for this assignment. You will not need to modify or add anything to Part 1. Part 2 is where your solution begins. 

**Part 1: Background.**
- 1A. Environment set-up 
- 1B. Data exploration 
- 1C. Tips 

**Part 2: Your implementation.** This is where you will implement your solution by modifying the following four functions within the `NaiveBayesClassifier()` class: 
- `__init__()`
- `train()`
- `predict()`
- `get_prob_label_given_word()` 

**Part 3: Evaluation on real datasets.** In the third part, you will evaluate your NaiveBayesClassifier on two real-world datasets. 
- 3A. You will train and evaluate on the `triage` data. 
- 3B. You will inspect words with the highest predicted probability for each label. 
- 3C. You will evaluate your trained classifier on the `covid` dataset. You will have to <span style="color:blue">answer a free-response question</span> about your classifier on this dataset. 
- 3D. Ethical considerations: You will <span style="color:blue">answer a free-response question</span> about the ethics of using your classifier. 

**(Optional) Part 4: Extra Credit.** Extra credit can only help you and will not hurt you. At the end of the semester, if you have a borderline grade, your grade could be increased given your efforts on extra credit. This section is intended to be open-ended and challenge you. We suggest you only attempt this section after you have completed all other parts and are satisifed with your submission. 

**Addtional instructions.** 
- Your submitted solution and code must be yours alone. Copying and pasting a solution from the internet or another source is considered a violation of the honor code. 
- However, you can talk to classmates about *high-level* approaches. In the **Process Reporting** section, record the names of any classmates you spoke with about this assignment. 

**Evaluation.**

For HW1, your solution was evaluated on F1, the harmonic mean of precision and recall. That metric is still important for text classification; however, on this assignment you will be evaluated on `accuracy`, the number of correct predictions divided by the total number of predictions. 

From your classifier's predictions, we count the number of `true positives (tp)`, `false positives (fp)`, `false negatives (fn)` and `true negatives (tn)` and calculate `accuracy` as 
$$ \text{accuracy} = \frac{tp+tn}{tp+tn+fp+fn} $$

Unlike rule-based systems (e.g., regular expressions) in which perfect accuracy is within rearch, it is often difficult to obtain perfect accuracy with text classifiers. 

Our reference implementation of Naive Bayes achieves the following `accuracy` scores on the `triage` dataset splits:  
- train : `0.829`
- dev: `0.733`
- test: `0.763`

**Grading.**

- **20 points (autograded):** This portion of your grade reflects how well your submission performs on the `training set` of the `triage` dataset compared to our reference implementation metrics. Your points are calculated as 
    ```
    (1-(0.829 - min(accuracy on train, 0.829))/0.829) * 20 points 
    ``` 
    
- **20 points (autograded):** This portion of your grade reflects how well your submission performs on the `dev set` of the `triage` dataset compared to our reference implementation metrics.  Your points are calculated as 
    ```
    (1 -(0.733 - min(accuracy on dev, 0.733))/0.733) * 20 points 
    ```
- **20 points (autograded):** This portion of your grade reflects how well your submission performs on the `test set` of the `triage` dataset compared to our reference implementation metrics. You will not have access to the test set but will be able to see your score on Gradescope. Your points are calculated as   
    ```
    (1-(0.763 - min(accuracy on test, 0.763))/0.763) * 20 points 
    ``` 
    
- **10 points (autograded):** The autograder will randomly sample words from the training corpus and evaluate your  `get_prob_label_given_word()`. 
- **5 points (manually graded):** TAs and the instructor will evaluate your response to Part 3C. 
- **5 points (manually graded):** TAs and the instructor will evaluate your response to Part 3D. 

**Submission.** 
Once you have completed Parts 1, 2 and 3, run the final cell in this notebook. This will create `submission.zip` which you will then upload to Gradescope. 

## 1A. Environment set-up

If you set-up your conda environment correctly in HW0, you should see `Python [conda env:cs375]` as the kernel in the upper right-hand corner of the Jupyter webpage you are currently on. Run the cell below to make sure your environment is correctly installed. 

In [1]:
# Environment check 
# Return to HW0 if you run into errors in this cell 
# Do not modify this cell 
import os
assert os.environ['CONDA_DEFAULT_ENV'] == "cs375"

import sys
assert sys.version_info.major == 3 and sys.version_info.minor == 8

If there are any errors after running the cell above, return to the instructions from `HW0`. If you are still having difficulty, reach out to the instructor or TAs via Piazza. 

In [2]:
# Import necessary modules for this assignment 
# Do not modify this cell 
from collections import defaultdict, Counter
import operator
import random
from typing import List, Dict, Union
import numpy as np
import matplotlib.pyplot as plt

from util import * #helper functions for this assignment located in util.py

**Note:** In this assignment, you are **NOT** allowed to import or use any other packages outside the Python standard and the ones we imported above.

This means you should not use `spaCy`, `NLTK`, `gensim`, or `scikit-learn`, even though those are provided in the conda environment we set up for you. If your solution uses any such extra dependencies it will fail the autograder.

## 1B. Data exploration 

As is typical when dealing with real-world data, the first thing we need to do is understand and characterize our data. For the `triage` dataset in this assignment, we provide you with approximately `26K` documents from several major disasters: 
- [Earthquake in Haiti (2010)](https://en.wikipedia.org/wiki/2010_Haiti_earthquake)

- [Floods in Pakistan (2010)](https://en.wikipedia.org/wiki/2010_Pakistan_floods)

- [Earthquake in Chile (2010)](https://en.wikipedia.org/wiki/2010_Chile_earthquake)

- [Hurricane Sandy in North America (2012)](https://en.wikipedia.org/wiki/Hurricane_Sandy)

Recall, our formulation of Naive Bayes from lecture uses $(x, y)$ pairs in which $x$ are documents and $y$ are class labels. 

For the `triage` dataset, these variables are as follows
- *x:* The documents we will classify are either text messages, Twitter posts, or snippets from news articles during the disasters above. 
    - These have all been translated from their original language to English by humans. 
    - However, the translations are not perfect and we might have to work with "messy" data as we would encounter in real-world settings. 
    - If you are curious about the crowdsourcing translation effort for messages from Haiti in particular, feel free to check out [this paper](https://nlp.stanford.edu/pubs/munro2010translation.pdf).
- *y:* Our class labels are annotations from humans from a crowdsourcing platform called CrowdFlower. 
    - *y=0* indicates this is a document that is not about aid, `not aid` 
    - *y=1* indicates this is a document about `aid`

In [3]:
# Let's load this data via the function from util.py
triage_dataset = load_data("./data/triage")

In [4]:
# This is a custom-defined Dataset class, defined in util.py 
print(type(triage_dataset))

<class 'util.Dataset'>


Both the `triage` and `covid` datasets are split into `train` and `dev` sets. Your solution will also be evaluated on a held-out `test` set via Gradescope, but as in real deployment settings you will not have access to the individual examples in the `test` set. 

In [5]:
print(f"triage_dataset.train contains {len(triage_dataset.train)} examples")

triage_dataset.train contains 21046 examples


In [6]:
print(f"triage_dataset.dev contains {len(triage_dataset.dev)} examples")

triage_dataset.dev contains 2573 examples


Let's look at individual examples, which are from the custom class `Example` from `util.py`. 

In [7]:
print(type(triage_dataset.train[0]))

<class 'util.Example'>


In [21]:
# Re-run this cell several times to look at different examples 
random_index = random.randint(0, len(triage_dataset.train))
print(f"Training example {random_index}:")
print(f"Label(y) = {triage_dataset.train[random_index].label}")
tokens = triage_dataset.train[random_index].words
text = " ".join(tokens)
print(f"Text = {text}")
print()
print(f"Tokens = {tokens}")

Training example 11107:
Label(y) = 1
Text = these were deployed in teams of five persons to provide minor surgical and primary healthcare in the area of destroyed rural health facilities

Tokens = ['these', 'were', 'deployed', 'in', 'teams', 'of', 'five', 'persons', 'to', 'provide', 'minor', 'surgical', 'and', 'primary', 'healthcare', 'in', 'the', 'area', 'of', 'destroyed', 'rural', 'health', 'facilities']


*Sanity check:* Does it make sense to you that the text above is about `aid` if `Label(y) = 1` or not about aid if `Label(y) = 0`? 

*Note:* The data you are given has already been preprocessed and tokenized -- all punctuation has been removed except for hashtags and apostrophes, and all text has been converted to lowercase. There is no need for any additional preprocessing on the text. In real-world applications, you almost always will have to implement these pre-processing steps yourself. 

## 1C. Tips 

Here, we provide some suggestions and hints. It is possible to implement a working solution without following these suggestions, so feel free to ignore the ones that are not relevant to your approach. 

- **Use log probabilities.** As we discussed in class, implementing everything in log space can help avoid underflow. 
- **Keep track of the vocabulary.** For the purposes of implementing Laplace Smoothing (+1 smoothing), it may be helpful to keep track of the vocabulary, the set of all words you've seen in the training data. The Python `set()` structure may be useful here.
- **Remember defaultdict and Counter.** You may find Python's [defaultdict](https://docs.python.org/3.8/library/collections.html#collections.defaultdict) and/or [Counter](https://docs.python.org/3.8/library/collections.html#collections.Counter) helpful in your implementation when counting.
- **In Python, assignment is by reference.** Remember that in Python, assignment is by reference, not by value, for non-primitive types. More simply, when you're assigning an existing list or dict to a new variable, it does NOT make a copy. It just gives a reference to the existing list or dict.
- **Unknown words in dev and test.** Words in the dev/test set that are not seen in your training set should be ignored in your Naive Bayes computations. 
- **Code length.** Our reference implementation is just under 100 lines of code, including the skeleton code. It's quite possible that you can make a working implementation in fewer lines. However, if your implementation is signficantly longer than 100 lines, that might be a sign that your implementations is more complicated than necessary. 
- **In-class implementation.** In may prove helpful to return to the language modeling code implementation we discussed together in class to think about some best practices in data structures for NLP.

## 2. Your solution 

Complete the implementation of the `NaiveBayesClassifier` below.

You are welcome to create additional helper functions *within* the class if your implementation requires it. However, any functions you write outside of the `NaiveBayesClassifier` class cannot be accessed by the autograder and may cause it to fail. 

In [ ]:
# TODO: complete the implementation below 
class NaiveBayesClassifier:
    """
    Implements Naive Bayes Classifier 
    Includes Laplace smoothing (add-1 smoothing) during training 
    """
    def __init__(self):
        # TODO: add other data structures needed for predict() or train()
        # CODE START
        pass 
        # CODE END

    def train(self, examples: List[Example]) -> None:
        """
        This function inputs a list of labeled examples and 
        trains the classifier via Naive Bayes.  
        
        Hints: 
            - Remember to use Laplace smoothing! 
        """
        # TODO: implement your solution here 
        # CODE START
        raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
        # CODE END
             
    def predict(self, examples: List[Example]) -> List[int]:
        """
        This function inputs a list of Examples and 
        predicts their labels using the learned classifier 
        
        It returns as a list of int variables that are the predicted
        labels (e.g. 0 or 1)
            
        Hints: 
            - Remember to use logs to prevent underflow!
            - Remember to ignore words not seen during training 
        """
        # TODO: implement your solution here 
        # CODE START
        raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
        # CODE END

    def get_prob_label_given_word(self, label: int) -> Dict:
        """
        This function returns a dictionary for which 
            - keys are each unigram word in the vocabulary  
            - values are p(label|word) from the trained model where
                label is the exact value of the label inputted as an argument above 
            
        Note: this is NOT the same as p(word|label)
        
        Hint: You should have already calculated and stored p(word|label) and p(label). 
            How can you use Bayes rule to obtain p(label|word)? 
        """
        # TODO: implement your solution here 
        # CODE START
        raise NotImplementedError("Solution not yet implemented!") #delete this line and add your solution
        # CODE END

#### Debugging on "toy" corpus 

Like most real-world NLP systems, it can be helpful to examine the correctness of our code on a small "toy" dataset that we can analytically calculate the answers for. We'll give you one here but in future assignments you'll develop these yourself. 

In [ ]:
# Toy corpus (subset of the example we used during lecture)
toks1 = ['fun', 'couple', 'love', 'love']
label1 = 0 
ex1 = Example(toks1, label1)

toks2 = ['fast', 'furious', 'shoot']
label2 = 1 
ex2 = Example(toks2, label2)

toks3 = ['couple', 'fly', 'fast', 'fun', 'fun']
label3 = 0 
ex3 = Example(toks3, label3)

toks4 = ['fast', 'couple', 'shoot', 'fly', 'bomb']
label4 = 1
ex4 = Example(toks4, label4)

toy_training_data = [ex1, ex2, ex3]
toy_dev_data = [ex4]

In [ ]:
# Call to check your implementation 
nb_classifier = NaiveBayesClassifier()
nb_classifier.train(toy_training_data)

After training, you may want to examine some of the data structures you created in `__init__()` within your implementation of your `NaiveBayesClassifier()` to ensure it's correct. Feel free to add cells below to check this. 

In [ ]:
# Call to check your predict implementation 
nb_classifier.predict(toy_dev_data)

Many students encounter errors in the cell above. 

*Hint:* If you have an error above, think about how you're handling the train and test vocabularies. 

In [ ]:
#Prints the p(y=0|word) for every word in the vocab 
y0_probs = nb_classifier.get_prob_label_given_word(0)
sorted(y0_probs.items(), key=lambda kv: -kv[1])

In [ ]:
y1_probs = nb_classifier.get_prob_label_given_word(1)
sorted(y1_probs.items(), key=lambda kv: -kv[1])

*Hint:* For `y0_probs` and `y1_probs` above, what should each word sum to?  Try testing this yourself. 

## 3. Evaluation 

#### 3A. Accuracy 
Let's evaluate the accuracy of your implementation on the `triage` dataset. Our reference implementation obtains: 
```
Accuracy (train): 0.82946878266654
Accuracy (dev): 0.7329965021375826
```

In [ ]:
# Load triage data (again)
triage_data = load_data("./data/triage")

In [ ]:
# NO NEED TO MODIFY THIS CELL 
# evaluate() is implemented in util.py 
# Inspecting it, you'll see it trains your classifier and then caculates accuracy 
nb_classifier = NaiveBayesClassifier()
evaluate(nb_classifier, triage_data)

In the cell above, our reference implementation also takes about `3 seconds` to run. If your code is running significantly slower, we recommend returning to your implementation and thinking about how you might speed it up. 

#### 3B. Validation via `P(label|word)`

When building NLP systems, it's often helpful to manually inspect aspects of your model and check if these match your intution about the problem.  

Let's use `get_prob_label_given_word()` to examine your model's predictions for the words with the highest `p(label|word)` below. Do these make sense? 

The autograder will test your implementation on some additional test cases. 

In [ ]:
# NO NEED TO MODIFY THIS CELL 

print('Aid (y=1), most probable words')
print('==='*15)
vocab_probs_positive = nb_classifier.get_prob_label_given_word(1)
top_10 = sorted(vocab_probs_positive.items(), key=lambda kv: -kv[1])[:10]
for word, prob in top_10:
    print("{0:<18} prob = {1}".format(word, np.round(prob, 3)))

In [ ]:
# NO NEED TO MODIFY THIS CELL 

print('Not aid (y=0), most probable words')
print('==='*15)
vocab_probs_negative = nb_classifier.get_prob_label_given_word(0)
top_10 = sorted(vocab_probs_negative.items(), key=lambda kv: -kv[1])[:10]
for word, prob in top_10:
    print("{0:<18} prob = {1}".format(word, np.round(prob, 3)))

#### 3C. Generalization. 

Let's try to use the classifier you just trained on another text classification test: predicting messages related to `COVID` as positive or negative sentiment. 

This dataset consists of `reddit` comments on posts related to the `COVID-19` pandemic in 2020.

Here the y-labels consist of `0` for `negative` sentiment and `1` for `positive` sentiment.

*Warning:* Because this is data scraped from the internet, there may be explicit or hateful content. 

In [22]:
# NO NEED TO MODIFY THIS CELL 
# Load COVID dataset 
covid_data = load_data("./data/covid")

In [26]:
# NO NEED TO MODIFY THIS CELL 
# Re-run this cell several times to look at different examples 
random_index = random.randint(0, len(covid_data.dev))
print(f"COVID dev example {random_index}:")
print(f"Label(y) = {covid_data.dev[random_index].label}")
tokens = covid_data.dev[random_index].words
text = " ".join(tokens)
print(f"Text = {text}")
print()
print(f"Tokens = {tokens}")

COVID dev example 9767:
Label(y) = 1
Text = Considering how its going in Singapore yikesYoure right though Theres a very stoic presence bring shown but people are running around like chickens with the heads cut off behind the scenes

Tokens = ['Considering', 'how', 'its', 'going', 'in', 'Singapore', 'yikesYoure', 'right', 'though', 'Theres', 'a', 'very', 'stoic', 'presence', 'bring', 'shown', 'but', 'people', 'are', 'running', 'around', 'like', 'chickens', 'with', 'the', 'heads', 'cut', 'off', 'behind', 'the', 'scenes']


In [ ]:
# NO NEED TO MODIFY THIS CELL 
# We apply your trained classifier to this new data and
print("Accuracy on COVID (dev) =", calculate_accuracy(covid_data.dev, nb_classifier))

How well did your trained classifier do? If it didn't do well, why do you think it did not? What are possible steps forward? 

(There is no one "correct" answer for this question. We will evaluating your answer based on your thought process. We are expecting *at minimum* two complete sentences for full credit.) 

**Part 3C Answer:**

*DELETE AND PUT YOUR ANSWER HERE.*

#### 3D. Ethical considerations. 

We've seen there are two types of errors a classifier can make: `false positives` and `false negatives`. For different applications, these types of errors have different consequences. 

Thinking about the `triage` dataset and task holistically, which types of errors do you think would potentially cause more harm? Why? 

(There is no one "correct" answer for this question. We will evaluating your answer based on your thought process. We are expecting *at minimum* three complete sentences for full credit.) 

**Part 3D Answer:**

*DELETE AND PUT YOUR ANSWER HERE.*

## (Optional) 4. Extra Credit 

*Extra credit can only help you and will not hurt you. At the end of the semester, if you have a borderline grade, your grade could be increased given your efforts on extra credit. This section is intended to be open-ended and challenge you. We suggest you only attempt this section after you have completed all other parts and are satisifed with your submission.*

Above, we have implemented the most basic version of Naive Bayes. Try to extend it and make it better. Some suggestions to try: 
- Remove stopwords (e.g., `from nltk.corpus import stopwords`) 
- Change counts to indicators 
- Your implementation above only had to work for binary classification. Write some test cases on a toy corpus to make sure it works for multi-class classificiation. 
- Follow the suggestions in J&M Ch.4.4 and add a feature that is counted whenever a word from that sentiment lexicon occurs (for evaluation on the `covid` dataset). 
- Change from unigrams to other n-grams (e.g. bigrams or trigrams) 
- For [additive smoothing](https://en.wikipedia.org/wiki/Additive_smoothing), make the smoothing parameter (which we set as 1 in Laplace smoothing) a hyperparameter. Tune this parameter on the development set.  
- Lemmatize words first (e.g. `from nltk.stem import WordNetLemmatizer`) 

In [29]:
#TODO: implement a new version or an extension of your previous class
import nltk 

class ExtraNaiveBayesClassifier:
    pass

## Submission 

**Processing reporting.** Please record your answers to the questions below by writing directly in this Markdown cell.

If you talked with any of your classmates on this assignment please list their names here:

*DELETE AND PUT YOUR ANSWER HERE.*

Approximately how much time did you spend on this assignment:

*DELETE AND PUT YOUR ANSWER HERE.*

**Download zip.** Once you're satsified with your solution, save this file and run the cell below to automatically zip your file. This will produce `submission.zip` in the same folder as this file (same folder as `hw2.ipynb`). 

Submit `submission.zip` to Gradescope. 

*Note:* This script assumes that you have the `zip` utility installed and you can use `bash` on your system. If the cell below does not work you may need to zip your file manually. 

In [ ]:
%%bash

if [[ ! -f "./hw2.ipynb" ]]
then
    echo "WARNING: Did not find notebook in Jupyter working directory. Manual solution: go to File->Download .ipynb to download your notebok and other files, then zip them locally."
else
    echo "Found notebook file, creating submission zip..."
    zip -r submission.zip hw2.ipynb
fi